In [ ]:
!pip install transformers
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold

import gensim
import gensim.downloader as gensim_api

import transformers

import wget

import ast

In [ ]:
np.random.seed(365)

In [ ]:
pd.options.display.max_colwidth = 150

In [ ]:
url = 'https://raw.githubusercontent.com/Kukarkin/Digital-Student-Assistant-Recommender/master/data/translated_preprocessed_names.csv'
dtf = pd.read_csv(url, index_col=0)
dtf.head()

,0
0,Новая концепция для музея «Микула» в Нижнем Новгороде
1,Развитие туристического кластера в Кроноцком заповеднике
2,Создание мобильного приложения СППР для диагностики болезни Паркинсона
3,Гуманитарная политика России
4,Подкаст про современное высшее образование


In [ ]:
tags = pd.read_csv("translated_preprocessed_tags.csv")
tags = list(tags["0"])
tags = ast.literal_eval(str(tags))
lst_dics = tags

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained('DeepPavlov/bert-base-multilingual-cased-sentence', do_lower_case=True)
nlp = transformers.TFBertModel.from_pretrained('DeepPavlov/bert-base-multilingual-cased-sentence', from_pt=True)

In [ ]:
def utils_bert_embedding(txt, tokenizer, nlp):
    idx = tokenizer.encode(txt)
    idx = np.array(idx)[None,:]  
    embedding = nlp(idx)
    X = np.array(embedding[0][0][1:-1])
    return X

# Embedding the tags using BERT
lst_mean_vecs = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) 
                 for txt in lst_dics]

X = np.array(lst_mean_vecs)

In [ ]:
# Embedding the titles using BERT      
dic_y = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) for txt in dtf['0']]

In [ ]:
# Calculate cosine similarities
similarities = np.array([metrics.pairwise.cosine_similarity(X, y.reshape(1,-1)).T.tolist()[0] for y in dic_y]).T

In [ ]:
names = list(dtf['0'])

for i in range(len(similarities)):
    ### assign randomly if there is no similarity
    if sum(similarities[i]) == 0:
       similarities[i] = [0]*len(names)
       similarities[i][np.random.choice(range(len(names)))] = 1
    ### rescale so they sum = 1
    similarities[i] = similarities[i] / sum(similarities[i])

## classify the label with highest similarity score
predicted_prob = similarities

In [ ]:
a = np.array(names)
predicted = [a[np.argpartition(pred, -4)[-4:]] for pred in predicted_prob]

In [ ]:
fin = pd.DataFrame(predicted)
fin2 = pd.DataFrame(lst_dics)
result = pd.concat([fin2, fin], axis=1)

In [ ]:
result